### Test: Smaller Language Models that are able to run on a V100

In an attempt to generate better serch queries that a theoretical car dealer would use to find a vehicle, we started to run the biggest-possible language models on a V100 in order to see what they might be able to do for us in terms of query-generation.

1: Testing GPT-j-6B

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "EleutherAI/gpt-j-6B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto").cuda()

input_text = "Hello, my name is"
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0]))


/storage/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /storage/venv/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cpu.so
/storage/venv/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
ERROR: /storage/venv/bin/python: undefined symbol: cudaRuntimeGetVersion
CUDA SETUP: libcudart.so path is None
CUDA SETUP: Is seems that your cuda installation is not in your path. See https://github.com/TimDettmers/bitsandbytes/issues/85 for more information.
CUDA SETUP: CUDA version lower than 11 are currently not supported for LLM.int8(). You will be only to use 8-bit optimizers and quantization routines!!
CUDA SETUP

/storage/venv/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/storage/venv/lib/python3.11/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('"/root/.vscode-server/cli/servers/Stable-cb0c47c0cfaad0757385834bd89d410c78a856c0/server/out/nls.messages.json","locale"'), PosixPath('"en","defaultMessagesFile"'), PosixPath('{}}'), PosixPath('"en","osLocale"'), PosixPath('"en","resolvedLanguage"'), PosixPath('{"userLocale"'), PosixPath('"en","availableLanguages"')}
  warn(msg)
/storage/venv/lib/python3.11/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found 

Hello, my name is David and I am a recovering alcoholic. I am also a recovering addict. I am a recovering addict because I was addicted to alcohol. I am a recovering alcoholic because I was addicted to alcohol. I am a recovering addict because I was addicted to alcohol


2: Testing Falcon-7B-instruct (which looked better from a standart-input-prompt perspective.)

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "tiiuae/falcon-7b-instruct"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model with custom code (Flash Attention is not used by default on V100/older setups)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    trust_remote_code=True,
    device_map="auto"  # or just .cuda() if single-GPU
)

# Inference
prompt = "Explain quantum physics in simple terms."
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.8,
        top_p=0.95,
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/storage/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/storage/venv/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multip


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /storage/venv/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cpu.so
/storage/venv/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
ERROR: /storage/venv/bin/python: undefined symbol: cudaRuntimeGetVersion
CUDA SETUP: libcudart.so path is None
CUDA SETUP: Is seems that your cuda installation is not in your path. See https://github.com/TimDettmers/bitsandbytes/issues/85 for more information.
CUDA SETUP: CUDA version lower than 11 are currently not supported for LLM.int8(). You will be only to use 8-bit optimizers and quantization routines!!
CUDA SETUP

Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.24s/it]
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Explain quantum physics in simple terms.
Quantum physics is a branch of physics that studies the behavior of tiny particles, like electrons and quarks, and how they interact with each other and with their environment. It's based on the idea that particles can exist in multiple states or locations at the same time until they're measured or observed. This is in contrast to classical physics, where objects are considered to have a definite position and momentum until an event occurs. In quantum physics, probabilities of particles' locations and behaviors are represented by wave functions, and interactions between particles are described by mathematical formulas involving matrices. These concepts are often counterintuitive and require significant mathematical sophistication.


After testing the models in-general we tried long to design a prompt that makes the small flcon model do what we want ... but the generated query was never really of high quality.

Step 1: Loading the model once more

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "tiiuae/falcon-7b-instruct"
CACHE_DIR = "/storage/courses/ds_workflow/car-web-scraping/model_cache"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=CACHE_DIR,)

# Load model with custom code (Flash Attention is not used by default on V100/older setups)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    trust_remote_code=True,
    cache_dir=CACHE_DIR,
    device_map="auto"  # or just .cuda() if single-GPU
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.55s/it]


Step 2: Generating a query that is supposed to match a specific vehicle from our .yaml

In [4]:
# Inference
prompt = """
You are a car dealer interested in a car. You want to know from a chatbot what car could match your description.
The search query shall be designed to match vaguely this result: 

    Category: Estate, 5 door
    Colour: red
    Emission class: EURO 5
    Engine type: Diesel
    Federal Motor Transport Authority (KBA) Key Manufacturer: '4136'
    Federal Motor Transport Authority (KBA) Key Type: AMY
    First registration: '01.2011'
    Fuel type: Diesel
    Location: D-76
    Power output: 125 KW / 170 PS
    Read mileage: 268.300 Kilometres
    Supplier: 'Subsidiary/authorised dealer of the brand(s): Audi'
    Total number of owners: 1 Owner
    Transmission: 6-gear manual transmission
    Vehicle Identification No.: ZAR93900007******
    Vehicle release: Release of the vehicle will take place at the earliest 5 working
      days following receipt of payment.

Please create a realistic search query that doesn't contain every detail but only a few of the provided ones.
Don't use exact numeric details but spans (i.e.: less than 300.000 Kilometres mileage).
Only choose a few sensible details to include them in the query. Make the query multiple sentences long.
Query of a car dealer:
"""
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.8,
        top_p=0.95,
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



You are a car dealer interested in a car. You want to know from a chatbot what car could match your description.
The search query shall be designed to match vaguely this result: 

    Category: Estate, 5 door
    Colour: red
    Emission class: EURO 5
    Engine type: Diesel
    Federal Motor Transport Authority (KBA) Key Manufacturer: '4136'
    Federal Motor Transport Authority (KBA) Key Type: AMY
    First registration: '01.2011'
    Fuel type: Diesel
    Location: D-76
    Power output: 125 KW / 170 PS
    Read mileage: 268.300 Kilometres
    Supplier: 'Subsidiary/authorised dealer of the brand(s): Audi'
    Total number of owners: 1 Owner
    Transmission: 6-gear manual transmission
    Vehicle Identification No.: ZAR93900007******
    Vehicle release: Release of the vehicle will take place at the earliest 5 working
      days following receipt of payment.

Please create a realistic search query that doesn't contain every detail but only a few of the provided ones.
Don't use exac

#### Side-Note: Requirements:

**Core compatible versions**  
torch==2.0.1+cu118     # Stable and compatible with CUDA 11.8  
transformers==4.30.2   # Works with older models without Triton/FlashAttention2  
accelerate==0.21.0     # Compatible with this transformers version  
sentencepiece==0.1.99  # Required for some tokenizer models  
scipy==1.11.4  
numpy==1.24.4  
tokenizers==0.13.3  
datasets==2.14.5  
peft==0.4.0            # Optional, for PEFT/LoRA if needed  
bitsandbytes==0.39.0   # Last version that can be compiled with CUDA 11.8 (optional)

**Hugging Face Hub**  
huggingface-hub==0.17.3  

**Optional utilities**  
tqdm==4.66.1  
einops==0.6.1